# IA 340 Mini Project

## Research Question

Can a non-fully electric car's price be accurately predicted by engine displacement, horsepower, top speed, and performance?

## Dataset Documentation

https://www.kaggle.com/datasets/abdulmalik1518/cars-datasets-2025?resource=download (Kaggle.com)

This is a dataset with information about a wide variety of cars. It covers brand, model, engine type, engine displacement, horsepower, top speed, performance (0-100 kmh), price, fuel type, seats, ad torque. 

Time period: 2025

It directly relates to the research question due to having the essential information for key aspects of cars, and listing the price of the cars. I am also extremely interested in cars, in whic I have some familiarity with this kind of data. 


## Import Data

In [47]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd

# Use latin1 encoding instead of utf-8
df = pd.read_csv('s3://kdr-ia-340-2025/Cars Datasets 2025.csv', encoding='latin1')

# Check first few rows
df.head()


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


## Data Cleaning

In [48]:
import pandas as pd
import numpy as np
import re

df.columns = df.columns.str.strip().str.replace(r'\s+', '_', regex=True)
df.head()

,Company_Names,Cars_Names,Engines,CC/Battery_Capacity,HorsePower,Total_Speed,Performance(0_-_100_)KM/H,Cars_Prices,Fuel_Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


Standardizing the column names. 

In [49]:
#drop electric-only vehicles
df = df[~df['Fuel_Types'].str.lower().str.contains('electric', na=False)]

Don't need to look at electric cars given the context of the question, so might as well drop it. 

In [50]:
#define function for eliminating ranges
def extract_number(value):
    """Extract the first numeric value (lower number if a range)."""
    if pd.isna(value):
        return np.nan
    value = str(value)
    numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', value)
    if not numbers:
        return np.nan
    num = numbers[0]
    return float(num.replace(',', ''))

#apply function
cols_to_clean = [
    'CC/Battery_Capacity',
    'HorsePower',
    'Total_Speed',
    'Performance(0_-_100_)KM/H',
    'Cars_Prices',
    'Seats',
    'Torque'
]

for col in cols_to_clean:
    df[col] = df[col].apply(extract_number)

df = df.dropna()

df.head()

,Company_Names,Cars_Names,Engines,CC/Battery_Capacity,HorsePower,Total_Speed,Performance(0_-_100_)KM/H,Cars_Prices,Fuel_Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990.0,963.0,340.0,2.5,1100000.0,plug in hyrbrid,2.0,800.0
1,ROLLS ROYCE,PHANTOM,V12,6749.0,563.0,250.0,5.3,460000.0,Petrol,5.0,900.0
2,Ford,KA+,1.2L Petrol,1200.0,70.0,165.0,10.5,12000.0,Petrol,5.0,100.0
3,MERCEDES,GT 63 S,V8,3982.0,630.0,250.0,3.2,161000.0,Petrol,4.0,900.0
4,AUDI,AUDI R8 Gt,V10,5204.0,602.0,320.0,3.6,253290.0,Petrol,2.0,560.0


The code above is cleaning the values in the chart that are ranges, such as the car price for the Ford KA+ being 12,000-15,000. the code splits this into two values with the given numbers of the range, and replaces the full range with the lower value. Ranges for any of the columns are generally for different upgrades/added features to the cars, so the lower number will reflect the base model car and thus give us the most reliable data. The function is being defined first, and then the function is applied to each of the columns. Also, any values with NaN or missing values are dropped at the end of it. 

In [51]:
#Final Cleaning
df.rename(columns={'Company_Names': 'Make'}, inplace=True)
df['Make'] = df['Make'].str.upper()


df.rename(columns={'CC/Battery_Capacity': 'Displacement(cc)'}, inplace=True)
df.rename(columns={'Performance(0_-_100_)KM/H': 'Performance(0-100_KMH)'}, inplace=True)
df.rename(columns={'Cars_Names': 'Model'}, inplace=True)
df.rename(columns={'Total_Speed': 'Top_Speed(KMH)'}, inplace=True)

df = df.sort_values(by=['Make', 'Model'])


df.head(20)

,Make,Model,Engines,Displacement(cc),HorsePower,Top_Speed(KMH),Performance(0-100_KMH),Cars_Prices,Fuel_Types,Seats,Torque
943,ACURA,ILX Premium,I4,2400.0,201.0,200.0,6.6,30000.0,Petrol,5.0,180.0
956,ACURA,ILX Special Edition,Inline-4,2400.0,201.0,200.0,6.6,30000.0,Petrol,5.0,180.0
952,ACURA,ILX Technology Package,Inline-4,2400.0,201.0,200.0,6.6,30000.0,Petrol,5.0,180.0
963,ACURA,ILX Technology Plus,Inline-4,2400.0,201.0,200.0,6.6,28000.0,Petrol,5.0,180.0
941,ACURA,MDX Advance,V6,3500.0,290.0,190.0,6.0,60000.0,Petrol,7.0,362.0
959,ACURA,MDX Elite,V6,3500.0,290.0,190.0,6.0,58000.0,Petrol,7.0,362.0
950,ACURA,MDX Hybrid,V6 Hybrid,3500.0,321.0,190.0,6.0,55000.0,Hybrid,7.0,335.0
946,ACURA,MDX SH-AWD,V6,3500.0,290.0,190.0,6.0,55000.0,Petrol,7.0,362.0
957,ACURA,MDX SH-AWD w/Technology Package,V6,3500.0,290.0,190.0,6.0,55000.0,Petrol,7.0,362.0
965,ACURA,MDX Sport Hybrid,V6 Hybrid,3500.0,321.0,190.0,6.0,52000.0,Hybrid,7.0,335.0


Finishing touches for cleaning the data and ordering it by make and model. 

## Process/Aggregate